# AILib Tutorial 9: Advanced Features

Explore advanced features and patterns in AILib. This tutorial covers:

- Async operations
- Streaming responses
- Custom LLM clients
- Advanced prompt engineering
- Performance optimization
- Integration patterns
- Error handling strategies
- Pydantic validation for type safety

## Setup

Import necessary modules:

In [ ]:
import asyncio
import time
from typing import AsyncIterator, Iterator, Optional
from concurrent.futures import ThreadPoolExecutor
import json

from ailib import OpenAIClient, LLMClient
from ailib.prompts import PromptTemplate, PromptBuilder
from ailib.chains import Chain
from ailib.agents import Agent, Tool, ToolRegistry, tool
from ailib import Session

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Create clients
client = OpenAIClient()
streaming_client = OpenAIClient(stream=True)

print("Ready for advanced features!")

## Async Operations

AILib supports async operations for better performance:

In [ ]:
# Create an async-compatible client
class AsyncOpenAIClient(OpenAIClient):
    """OpenAI client with async support."""
    
    async def acompletion(self, prompt: str, **kwargs) -> str:
        """Async completion."""
        # In production, use actual async OpenAI client
        # This is a simulation
        await asyncio.sleep(0.1)  # Simulate API delay
        return self.complete(prompt, **kwargs)
    
    async def achat(self, messages: list, **kwargs) -> str:
        """Async chat."""
        await asyncio.sleep(0.1)  # Simulate API delay
        return self.chat(messages, **kwargs)

# Async batch processing
async def process_batch_async(prompts: list[str]) -> list[str]:
    """Process multiple prompts concurrently."""
    async_client = AsyncOpenAIClient()
    
    # Create tasks for all prompts
    tasks = [async_client.acompletion(prompt) for prompt in prompts]
    
    # Wait for all to complete
    results = await asyncio.gather(*tasks)
    
    return results

# Test async processing
async def test_async():
    prompts = [
        "Write a haiku about Python",
        "Explain quantum computing in one sentence",
        "What is the meaning of life?",
        "Describe the color blue to a blind person"
    ]
    
    # Time sync processing
    start_sync = time.time()
    sync_results = [client.complete(p) for p in prompts]
    sync_time = time.time() - start_sync
    
    # Time async processing
    start_async = time.time()
    async_results = await process_batch_async(prompts)
    async_time = time.time() - start_async
    
    print(f"Sync processing time: {sync_time:.2f}s")
    print(f"Async processing time: {async_time:.2f}s")
    print(f"Speedup: {sync_time/async_time:.2f}x")
    
    print("\nResults:")
    for i, (prompt, result) in enumerate(zip(prompts, async_results)):
        print(f"\n{i+1}. {prompt}")
        print(f"   {result[:100]}...")

# Run async test
await test_async()

## Streaming Responses

Handle streaming responses for real-time output:

In [ ]:
# Streaming response handler
class StreamHandler:
    """Handle streaming responses with callbacks."""
    
    def __init__(self):
        self.tokens = []
        self.metadata = {}
    
    def on_token(self, token: str):
        """Called for each token."""
        self.tokens.append(token)
        print(token, end='', flush=True)
    
    def on_complete(self):
        """Called when streaming completes."""
        self.metadata['total_tokens'] = len(self.tokens)
        self.metadata['full_response'] = ''.join(self.tokens)
        print("\n\n[Streaming complete]")
    
    def get_response(self) -> str:
        """Get the full response."""
        return ''.join(self.tokens)

# Stream with progress tracking
def stream_with_progress(prompt: str, max_tokens: int = 200):
    """Stream response with progress tracking."""
    handler = StreamHandler()
    
    print("Streaming response:")
    print("=" * 50)
    
    # Get streaming response
    stream = streaming_client.complete(prompt)
    
    # Process stream
    for token in stream:
        if token:
            handler.on_token(token)
    
    handler.on_complete()
    
    # Show metadata
    print(f"\nTotal tokens: {handler.metadata['total_tokens']}")
    
    return handler.get_response()

# Test streaming
response = stream_with_progress(
    "Write a short story about a robot learning to paint (max 100 words):"
)

# Advanced streaming with filtering
class FilteredStreamHandler(StreamHandler):
    """Stream handler with content filtering."""
    
    def __init__(self, filters: list[str] = None):
        super().__init__()
        self.filters = filters or []
        self.filtered_count = 0
    
    def on_token(self, token: str):
        """Filter tokens before processing."""
        # Check filters
        for filter_word in self.filters:
            if filter_word.lower() in token.lower():
                token = "[FILTERED]"
                self.filtered_count += 1
                break
        
        super().on_token(token)

print("\n\nFiltered streaming example:")
print("=" * 50)

# This is just an example - in production, use proper content moderation
filtered_handler = FilteredStreamHandler(filters=["robot", "paint"])
# Process with filtering (mock example)
print("[Filtered streaming would replace sensitive words]")

## Custom LLM Clients

Create custom LLM clients for specific providers:

In [ ]:
# Custom LLM client example
class CustomLLMClient(LLMClient):
    """Custom LLM client implementation."""
    
    def __init__(self, endpoint: str, api_key: str, **kwargs):
        super().__init__(**kwargs)
        self.endpoint = endpoint
        self.api_key = api_key
        self.request_count = 0
        self.cache = {}
    
    def complete(self, prompt: str, **kwargs) -> str:
        """Custom completion implementation."""
        self.request_count += 1
        
        # Check cache
        cache_key = f"{prompt}:{json.dumps(kwargs, sort_keys=True)}"
        if cache_key in self.cache:
            return self.cache[cache_key]
        
        # Mock API call
        response = f"Custom response to: {prompt[:50]}..."
        
        # Cache response
        self.cache[cache_key] = response
        
        return response
    
    def chat(self, messages: list[dict], **kwargs) -> str:
        """Custom chat implementation."""
        # Convert messages to prompt
        prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])
        return self.complete(prompt, **kwargs)
    
    def get_stats(self) -> dict:
        """Get client statistics."""
        return {
            "requests": self.request_count,
            "cache_size": len(self.cache),
            "cache_hit_rate": len(self.cache) / max(self.request_count, 1)
        }

# Local model client
class LocalModelClient(LLMClient):
    """Client for local models (e.g., llama.cpp, GGML)."""
    
    def __init__(self, model_path: str, **kwargs):
        super().__init__(**kwargs)
        self.model_path = model_path
        # In production, load actual model here
        self.model_loaded = True
    
    def complete(self, prompt: str, **kwargs) -> str:
        """Run local model inference."""
        if not self.model_loaded:
            raise RuntimeError("Model not loaded")
        
        # Mock local inference
        # In production, use actual model inference
        return f"Local model response: {prompt[:30]}..."
    
    def chat(self, messages: list[dict], **kwargs) -> str:
        """Chat with local model."""
        # Format messages for local model
        formatted_prompt = self._format_messages(messages)
        return self.complete(formatted_prompt, **kwargs)
    
    def _format_messages(self, messages: list[dict]) -> str:
        """Format messages for local model."""
        formatted = []
        for msg in messages:
            if msg['role'] == 'system':
                formatted.append(f"System: {msg['content']}")
            elif msg['role'] == 'user':
                formatted.append(f"Human: {msg['content']}")
            elif msg['role'] == 'assistant':
                formatted.append(f"Assistant: {msg['content']}")
        
        formatted.append("Assistant:")  # Prompt for response
        return "\n\n".join(formatted)

# Test custom clients
custom_client = CustomLLMClient(
    endpoint="https://api.custom-llm.com",
    api_key="test-key"
)

# Make requests
response1 = custom_client.complete("Hello, world!")
response2 = custom_client.complete("Hello, world!")  # Should hit cache
response3 = custom_client.complete("Different prompt")

print("Custom Client Stats:")
print(json.dumps(custom_client.get_stats(), indent=2))

# Test local model client
local_client = LocalModelClient(model_path="/path/to/model.gguf")
local_response = local_client.chat([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is Python?"}
])
print(f"\nLocal model response: {local_response}")

## Advanced Prompt Engineering

Sophisticated prompt techniques:

In [ ]:
# Chain of Thought prompting
class ChainOfThoughtTemplate(PromptTemplate):
    """Template that enforces chain-of-thought reasoning."""
    
    def __init__(self, base_template: str, **defaults):
        # Wrap template with CoT instructions
        cot_template = f"""{base_template}

Let's think step by step:
1. First, I'll identify the key components of the problem
2. Then, I'll analyze each component
3. Finally, I'll synthesize a solution

Step-by-step reasoning:
"""
        super().__init__(cot_template, **defaults)

# Few-shot learning template
class FewShotTemplate:
    """Template for few-shot learning."""
    
    def __init__(self, task_description: str):
        self.task_description = task_description
        self.examples = []
    
    def add_example(self, input_text: str, output_text: str):
        """Add an example."""
        self.examples.append({"input": input_text, "output": output_text})
    
    def format(self, input_text: str) -> str:
        """Format with examples."""
        prompt_parts = [self.task_description, "\nExamples:\n"]
        
        for i, example in enumerate(self.examples, 1):
            prompt_parts.append(f"Example {i}:")
            prompt_parts.append(f"Input: {example['input']}")
            prompt_parts.append(f"Output: {example['output']}\n")
        
        prompt_parts.append("Now, process this input:")
        prompt_parts.append(f"Input: {input_text}")
        prompt_parts.append("Output:")
        
        return "\n".join(prompt_parts)

# Self-consistency prompting
class SelfConsistencyPrompt:
    """Generate multiple solutions and select the best."""
    
    def __init__(self, client: LLMClient, num_samples: int = 3):
        self.client = client
        self.num_samples = num_samples
    
    def generate(self, prompt: str) -> dict:
        """Generate multiple solutions."""
        solutions = []
        
        for i in range(self.num_samples):
            # Add variation to prompt
            varied_prompt = f"{prompt}\n\n(Solution {i+1}:)"
            solution = self.client.complete(varied_prompt, temperature=0.7)
            solutions.append(solution)
        
        # Analyze solutions
        analysis_prompt = f"""Here are {len(solutions)} solutions to a problem:

{chr(10).join([f'Solution {i+1}: {s}' for i, s in enumerate(solutions)])}

Which solution is best and why? Provide a brief analysis."""
        
        analysis = self.client.complete(analysis_prompt)
        
        return {
            "solutions": solutions,
            "analysis": analysis,
            "best_solution": solutions[0]  # In production, parse analysis
        }

# Test advanced prompting
print("Chain of Thought Example:")
print("=" * 50)

cot_template = ChainOfThoughtTemplate(
    "Solve this problem: {problem}"
)

cot_response = client.complete(
    cot_template.format(problem="If a train travels 120 miles in 2 hours, how far will it travel in 5 hours at the same speed?")
)
print(cot_response)

print("\n\nFew-Shot Learning Example:")
print("=" * 50)

few_shot = FewShotTemplate("Convert natural language to SQL queries:")
few_shot.add_example(
    "Show all users",
    "SELECT * FROM users;"
)
few_shot.add_example(
    "Find users named John",
    "SELECT * FROM users WHERE name = 'John';"
)
few_shot.add_example(
    "Count active users",
    "SELECT COUNT(*) FROM users WHERE status = 'active';"
)

sql_prompt = few_shot.format("Get emails of premium users")
sql_response = client.complete(sql_prompt)
print(f"Generated SQL: {sql_response}")

print("\n\nSelf-Consistency Example:")
print("=" * 50)

self_consistency = SelfConsistencyPrompt(client, num_samples=3)
result = self_consistency.generate(
    "What's the best way to learn programming?"
)

print("Multiple solutions generated:")
for i, solution in enumerate(result['solutions'], 1):
    print(f"\nSolution {i}: {solution[:100]}...")

print(f"\nAnalysis: {result['analysis'][:200]}...")

## Performance Optimization

Techniques for optimizing AILib applications:

In [ ]:
# Response caching
class CachedClient(LLMClient):
    """LLM client with intelligent caching."""
    
    def __init__(self, base_client: LLMClient, cache_size: int = 100):
        self.base_client = base_client
        self.cache = {}
        self.cache_size = cache_size
        self.hits = 0
        self.misses = 0
    
    def _get_cache_key(self, prompt: str, **kwargs) -> str:
        """Generate cache key."""
        # Include relevant kwargs in key
        key_parts = [prompt]
        for k in ['temperature', 'max_tokens', 'model']:
            if k in kwargs:
                key_parts.append(f"{k}:{kwargs[k]}")
        return "|".join(key_parts)
    
    def complete(self, prompt: str, **kwargs) -> str:
        """Cached completion."""
        cache_key = self._get_cache_key(prompt, **kwargs)
        
        if cache_key in self.cache:
            self.hits += 1
            return self.cache[cache_key]
        
        self.misses += 1
        response = self.base_client.complete(prompt, **kwargs)
        
        # Manage cache size
        if len(self.cache) >= self.cache_size:
            # Remove oldest entry (simple FIFO)
            oldest = next(iter(self.cache))
            del self.cache[oldest]
        
        self.cache[cache_key] = response
        return response
    
    def chat(self, messages: list[dict], **kwargs) -> str:
        """Cached chat."""
        # Convert messages to string for caching
        prompt = json.dumps(messages)
        return self.complete(prompt, **kwargs)
    
    def get_cache_stats(self) -> dict:
        """Get cache statistics."""
        total = self.hits + self.misses
        return {
            "hits": self.hits,
            "misses": self.misses,
            "hit_rate": self.hits / total if total > 0 else 0,
            "cache_size": len(self.cache)
        }

# Batch processing optimization
class BatchProcessor:
    """Process multiple requests efficiently."""
    
    def __init__(self, client: LLMClient, batch_size: int = 5):
        self.client = client
        self.batch_size = batch_size
    
    def process_batch(self, items: list, prompt_template: PromptTemplate) -> list:
        """Process items in batches."""
        results = []
        
        for i in range(0, len(items), self.batch_size):
            batch = items[i:i + self.batch_size]
            
            # Process batch in parallel using threads
            with ThreadPoolExecutor(max_workers=self.batch_size) as executor:
                futures = []
                for item in batch:
                    prompt = prompt_template.format(item=item)
                    future = executor.submit(self.client.complete, prompt)
                    futures.append(future)
                
                # Collect results
                for future in futures:
                    results.append(future.result())
        
        return results

# Token optimization
class TokenOptimizer:
    """Optimize token usage."""
    
    @staticmethod
    def compress_prompt(prompt: str, max_length: int = 1000) -> str:
        """Compress prompt to save tokens."""
        if len(prompt) <= max_length:
            return prompt
        
        # Smart truncation - keep beginning and end
        start_length = max_length // 2
        end_length = max_length // 2
        
        return f"{prompt[:start_length]}... [truncated] ...{prompt[-end_length:]}"
    
    @staticmethod
    def estimate_tokens(text: str) -> int:
        """Estimate token count (rough approximation)."""
        # Rough estimate: ~1 token per 4 characters
        return len(text) // 4

# Test performance optimizations
print("Testing Cached Client:")
print("=" * 50)

cached_client = CachedClient(client)

# Make repeated requests
test_prompts = [
    "What is Python?",
    "What is JavaScript?",
    "What is Python?",  # Should hit cache
    "What is Java?",
    "What is Python?"   # Should hit cache
]

for prompt in test_prompts:
    start = time.time()
    response = cached_client.complete(prompt)
    elapsed = time.time() - start
    print(f"Prompt: {prompt} - Time: {elapsed:.3f}s")

print("\nCache Statistics:")
print(json.dumps(cached_client.get_cache_stats(), indent=2))

# Test batch processing
print("\n\nTesting Batch Processing:")
print("=" * 50)

batch_processor = BatchProcessor(client, batch_size=3)
items = [f"Item {i}" for i in range(10)]
template = PromptTemplate("Describe {item} in one word")

start = time.time()
results = batch_processor.process_batch(items, template)
elapsed = time.time() - start

print(f"Processed {len(items)} items in {elapsed:.2f}s")
print(f"Average time per item: {elapsed/len(items):.3f}s")

# Test token optimization
print("\n\nTesting Token Optimization:")
print("=" * 50)

long_prompt = "This is a very long prompt " * 100
compressed = TokenOptimizer.compress_prompt(long_prompt, max_length=100)

print(f"Original length: {len(long_prompt)} chars")
print(f"Original tokens (est): {TokenOptimizer.estimate_tokens(long_prompt)}")
print(f"Compressed length: {len(compressed)} chars")
print(f"Compressed tokens (est): {TokenOptimizer.estimate_tokens(compressed)}")
print(f"Token savings: {(1 - len(compressed)/len(long_prompt))*100:.1f}%")

## Integration Patterns

Integrate AILib with other systems:

In [ ]:
# Database integration
class DatabaseAgent:
    """Agent that interacts with databases."""
    
    def __init__(self, client: LLMClient):
        self.client = client
        self.tools = self._create_db_tools()
        self.agent = Agent(llm=client, tools=self.tools)
    
    def _create_db_tools(self) -> ToolRegistry:
        """Create database tools."""
        registry = ToolRegistry()
        
        @tool(registry=registry)
        def query_database(sql: str) -> list:
            """Execute SQL query (mock)."""
            # In production, use actual database connection
            mock_results = [
                {"id": 1, "name": "Alice", "role": "Engineer"},
                {"id": 2, "name": "Bob", "role": "Manager"}
            ]
            return mock_results
        
        @tool(registry=registry)
        def describe_table(table_name: str) -> dict:
            """Get table schema."""
            schemas = {
                "users": {
                    "columns": ["id", "name", "email", "role", "created_at"],
                    "primary_key": "id"
                },
                "orders": {
                    "columns": ["id", "user_id", "total", "status", "created_at"],
                    "primary_key": "id"
                }
            }
            return schemas.get(table_name, {"error": "Table not found"})
        
        return registry
    
    def analyze_data(self, request: str) -> str:
        """Analyze data based on natural language request."""
        return self.agent.run(request)

# API integration
class APIIntegration:
    """Integrate with external APIs."""
    
    def __init__(self, client: LLMClient):
        self.client = client
        self.api_specs = {}
    
    def register_api(self, name: str, spec: dict):
        """Register an API specification."""
        self.api_specs[name] = spec
    
    def generate_api_call(self, request: str) -> dict:
        """Generate API call from natural language."""
        # Build prompt with API specs
        prompt = f"""
Available APIs:
{json.dumps(self.api_specs, indent=2)}

User request: {request}

Generate the appropriate API call as JSON:
"""
        
        response = self.client.complete(prompt)
        
        # Parse response (in production, add error handling)
        try:
            api_call = json.loads(response)
            return api_call
        except:
            return {"error": "Failed to generate API call"}

# Event-driven integration
class EventDrivenAgent:
    """Agent that responds to events."""
    
    def __init__(self, client: LLMClient):
        self.client = client
        self.event_handlers = {}
        self.event_history = []
    
    def register_handler(self, event_type: str, handler_prompt: str):
        """Register an event handler."""
        self.event_handlers[event_type] = handler_prompt
    
    def handle_event(self, event: dict) -> str:
        """Handle an incoming event."""
        event_type = event.get('type')
        
        if event_type not in self.event_handlers:
            return "No handler for event type"
        
        # Get handler prompt
        handler_prompt = self.event_handlers[event_type]
        
        # Build context
        context = f"""
Event: {json.dumps(event, indent=2)}
Recent events: {json.dumps(self.event_history[-5:], indent=2)}

{handler_prompt}
"""
        
        response = self.client.complete(context)
        
        # Store event
        self.event_history.append({
            "event": event,
            "response": response,
            "timestamp": time.time()
        })
        
        return response

# Test integrations
print("Database Agent Example:")
print("=" * 50)

db_agent = DatabaseAgent(client)
result = db_agent.analyze_data(
    "Show me all users who are engineers"
)
print(f"Query result: {result}")

print("\n\nAPI Integration Example:")
print("=" * 50)

api_integration = APIIntegration(client)
api_integration.register_api("weather", {
    "endpoint": "/weather",
    "params": {"city": "string", "units": "string"}
})
api_integration.register_api("stocks", {
    "endpoint": "/stocks",
    "params": {"symbol": "string", "period": "string"}
})

api_call = api_integration.generate_api_call(
    "Get the weather in New York in celsius"
)
print(f"Generated API call: {json.dumps(api_call, indent=2)}")

print("\n\nEvent-Driven Agent Example:")
print("=" * 50)

event_agent = EventDrivenAgent(client)
event_agent.register_handler(
    "user_signup",
    "Generate a welcome message for the new user"
)
event_agent.register_handler(
    "error",
    "Analyze the error and suggest a solution"
)

# Handle events
signup_event = {
    "type": "user_signup",
    "user": {"name": "Alice", "email": "alice@example.com"}
}
response = event_agent.handle_event(signup_event)
print(f"Welcome message: {response}")

## Advanced Error Handling

Robust error handling strategies:

In [ ]:
# Retry with exponential backoff
class RetryClient(LLMClient):
    """Client with automatic retry logic."""
    
    def __init__(self, base_client: LLMClient, max_retries: int = 3):
        self.base_client = base_client
        self.max_retries = max_retries
    
    def complete(self, prompt: str, **kwargs) -> str:
        """Complete with retry."""
        last_error = None
        
        for attempt in range(self.max_retries):
            try:
                return self.base_client.complete(prompt, **kwargs)
            except Exception as e:
                last_error = e
                
                # Exponential backoff
                wait_time = 2 ** attempt
                print(f"Retry {attempt + 1}/{self.max_retries} after {wait_time}s...")
                time.sleep(wait_time)
        
        raise Exception(f"Failed after {self.max_retries} retries: {last_error}")
    
    def chat(self, messages: list[dict], **kwargs) -> str:
        """Chat with retry."""
        # Similar retry logic
        return self.complete(json.dumps(messages), **kwargs)

# Circuit breaker pattern
class CircuitBreakerClient(LLMClient):
    """Client with circuit breaker for fault tolerance."""
    
    def __init__(self, base_client: LLMClient, failure_threshold: int = 5):
        self.base_client = base_client
        self.failure_threshold = failure_threshold
        self.failure_count = 0
        self.is_open = False
        self.last_failure_time = None
        self.reset_timeout = 60  # seconds
    
    def _check_circuit(self):
        """Check if circuit should be reset."""
        if self.is_open and self.last_failure_time:
            if time.time() - self.last_failure_time > self.reset_timeout:
                print("Circuit breaker reset")
                self.is_open = False
                self.failure_count = 0
    
    def complete(self, prompt: str, **kwargs) -> str:
        """Complete with circuit breaker."""
        self._check_circuit()
        
        if self.is_open:
            raise Exception("Circuit breaker is open - service unavailable")
        
        try:
            response = self.base_client.complete(prompt, **kwargs)
            # Success - reset failure count
            self.failure_count = 0
            return response
        except Exception as e:
            self.failure_count += 1
            self.last_failure_time = time.time()
            
            if self.failure_count >= self.failure_threshold:
                print(f"Circuit breaker opened after {self.failure_count} failures")
                self.is_open = True
            
            raise e
    
    def chat(self, messages: list[dict], **kwargs) -> str:
        """Chat with circuit breaker."""
        return self.complete(json.dumps(messages), **kwargs)

# Fallback strategies
class FallbackClient(LLMClient):
    """Client with fallback options."""
    
    def __init__(self, primary_client: LLMClient, fallback_client: LLMClient):
        self.primary_client = primary_client
        self.fallback_client = fallback_client
    
    def complete(self, prompt: str, **kwargs) -> str:
        """Complete with fallback."""
        try:
            return self.primary_client.complete(prompt, **kwargs)
        except Exception as e:
            print(f"Primary client failed: {e}")
            print("Falling back to secondary client...")
            return self.fallback_client.complete(prompt, **kwargs)
    
    def chat(self, messages: list[dict], **kwargs) -> str:
        """Chat with fallback."""
        try:
            return self.primary_client.chat(messages, **kwargs)
        except Exception:
            return self.fallback_client.chat(messages, **kwargs)

print("Advanced Error Handling Examples:")
print("=" * 50)

# Demonstrate retry client
print("\nRetry Client (simulated):")
print("Would retry failed requests with exponential backoff")

# Demonstrate circuit breaker
print("\nCircuit Breaker (simulated):")
print("Would open circuit after 5 failures, preventing cascading failures")

# Demonstrate fallback
print("\nFallback Client (simulated):")
print("Would automatically switch to backup LLM if primary fails")

## Pydantic Validation

AILib provides comprehensive validation using Pydantic for type safety and data integrity:

In [ ]:
## Summary

In this tutorial, you learned advanced AILib features:

- ✅ Async operations for better performance
- ✅ Streaming responses for real-time output
- ✅ Creating custom LLM clients
- ✅ Advanced prompt engineering techniques
- ✅ Performance optimization strategies
- ✅ Integration patterns with external systems
- ✅ Robust error handling approaches
- ✅ Pydantic validation for type safety and data integrity

These advanced features enable you to:
- Build scalable AI applications
- Integrate with existing systems
- Handle edge cases gracefully
- Optimize for performance and cost
- Ensure data quality with validation

## Next Steps

Ready for the final tutorial?

- **Tutorial 10: Real-World Examples** - Complete applications using everything you've learned

Happy building! 🚀

## Summary

In this tutorial, you learned advanced AILib features:

- ✅ Async operations for better performance
- ✅ Streaming responses for real-time output
- ✅ Creating custom LLM clients
- ✅ Advanced prompt engineering techniques
- ✅ Performance optimization strategies
- ✅ Integration patterns with external systems
- ✅ Robust error handling approaches

These advanced features enable you to:
- Build scalable AI applications
- Integrate with existing systems
- Handle edge cases gracefully
- Optimize for performance and cost

## Next Steps

Ready for the final tutorial?

- **Tutorial 10: Real-World Examples** - Complete applications using everything you've learned

Happy building! 🚀